In [1]:
from datetime import date
import os
import pandas as pd
import math
import os
import statistics
from joblib import Parallel, delayed
from scipy import stats
import plotly.express as px
import single_cell_reloc_parquet.global_functions.global_variables as gv
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
plotly.io.kaleido.scope.mathjax = None #* This is an important line for timely svg output. This is to stop a call to the internet for prettier looking math
import kaleido
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context('paper')

print(plotly.__version__, kaleido.__version__)

# sns.set(rc = {'figure.figsize':(15,8)}) #* Set the figure size to be larger
# sns.set(rc = {'figure.figsize':(40,30)}) #* Set the figure size to be the shape of a powerpoint slide

5.13.0 0.2.1


In [2]:
pd.read_parquet("D:\ALL_FINAL\Combined_by_perc\ZPR1_selected.parquet")

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,mKO_RAWfactor_lower,<lambda_0>_y,count,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,192.099947,0.042893,9,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,192.099947,0.042893,9,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,192.099947,0.042893,9,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,192.099947,0.042893,9,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,192.099947,0.042893,9,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,d0216r1p050200c0123,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.105556,1.161111,1.188889,1.259833,1.159707,...,192.099947,0.031678,44,180.0,-0.033925,0.668127,-0.211716,180.0,0.664466,-0.257109
2005,d0216r1p050200c0127,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.130612,1.188776,1.222704,1.341786,1.182292,...,192.099947,0.031678,44,196.0,-0.061970,-0.210056,0.718936,196.0,-0.226961,0.703128
2006,d0216r1p050200c0131,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.086705,1.144509,1.187861,1.265896,1.132404,...,192.099947,0.031678,44,173.0,-0.033060,0.695204,-0.645200,173.0,0.692350,-0.677212
2007,d0216r1p050200c0134,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.111675,1.177665,1.213198,1.319797,1.173909,...,192.099947,0.031678,44,197.0,-0.054165,0.034340,0.774552,197.0,0.018615,0.763143


In [3]:
test = pd.read_parquet("D:\ALL_FINAL\Combined_by_perc\ZPR1_selected.parquet")

In [4]:
def reloc_yet_all(full_data, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy()
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])
	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	#< Added NOV23
	df_all_rem["Yet"] = 0
	df_all_rem["Does"] = 0
	df_all_rem["When"] = None

	df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna('backfill')
	df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].transform('max')

	working = df_all_rem.append(working)#. , ignore_index = True)

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [5]:
reloc_yet_all(test)

TypeError: reloc_yet_all() missing 2 required positional arguments: 'post_path' and 'output_file'

In [6]:
reloc_yet_all(test, "D:\ALL_FINAL\Combined_by_perc","TEST.parquet")

<ipython-input-4-135e6baf0c23>:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,0,NaN
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,0,NaN
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,0,NaN
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,0,NaN
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,d0216r1p050200c0123,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.105556,1.161111,1.188889,1.259833,1.159707,...,180.0,-0.033925,0.668127,-0.211716,180.0,0.664466,-0.257109,0,0,d0216r1p050200f0068
2005,d0216r1p050200c0127,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.130612,1.188776,1.222704,1.341786,1.182292,...,196.0,-0.061970,-0.210056,0.718936,196.0,-0.226961,0.703128,1,1,d0216r1p050200f0069
2006,d0216r1p050200c0131,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.086705,1.144509,1.187861,1.265896,1.132404,...,173.0,-0.033060,0.695204,-0.645200,173.0,0.692350,-0.677212,0,0,d0216r1p050200f0071
2007,d0216r1p050200c0134,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.111675,1.177665,1.213198,1.319797,1.173909,...,197.0,-0.054165,0.034340,0.774552,197.0,0.018615,0.763143,0,0,d0216r1p050200f0072


In [7]:
def reloc_yet_all(full_data, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy()
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])
	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	#< Added NOV23
	df_all_rem["Yet"] = 0
	df_all_rem["Does"] = 0
	df_all_rem["When"] = None

	df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].bfill()
	df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].transform('max')

	working = df_all_rem.append(working)#. , ignore_index = True)

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [8]:
reloc_yet_all(test, "D:\ALL_FINAL\Combined_by_perc","TEST.parquet")

<ipython-input-7-fec446aa6b8e>:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,0,NaN
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,0,NaN
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,0,NaN
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,0,NaN
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,d0216r1p050200c0123,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.105556,1.161111,1.188889,1.259833,1.159707,...,180.0,-0.033925,0.668127,-0.211716,180.0,0.664466,-0.257109,0,0,d0216r1p050200f0068
2005,d0216r1p050200c0127,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.130612,1.188776,1.222704,1.341786,1.182292,...,196.0,-0.061970,-0.210056,0.718936,196.0,-0.226961,0.703128,1,1,d0216r1p050200f0069
2006,d0216r1p050200c0131,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.086705,1.144509,1.187861,1.265896,1.132404,...,173.0,-0.033060,0.695204,-0.645200,173.0,0.692350,-0.677212,0,0,d0216r1p050200f0071
2007,d0216r1p050200c0134,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.111675,1.177665,1.213198,1.319797,1.173909,...,197.0,-0.054165,0.034340,0.774552,197.0,0.018615,0.763143,0,0,d0216r1p050200f0072


In [9]:
def reloc_yet_all(full_data, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy()
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])
	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	#< Added NOV23
	df_all_rem["Yet"] = 0
	df_all_rem["Does"] = None
	df_all_rem["When"] = None

	df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].bfill()
	df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].bfill()

	working = df_all_rem.append(working)#. , ignore_index = True)

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [10]:
reloc_yet_all(test, "D:\ALL_FINAL\Combined_by_perc","TEST.parquet")

<ipython-input-9-d9c743b9a06f>:45: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,NaN,NaN
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,NaN,NaN
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,NaN,NaN
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,NaN,NaN
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,d0216r1p050200c0123,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.105556,1.161111,1.188889,1.259833,1.159707,...,180.0,-0.033925,0.668127,-0.211716,180.0,0.664466,-0.257109,0,0,d0216r1p050200f0068
2005,d0216r1p050200c0127,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.130612,1.188776,1.222704,1.341786,1.182292,...,196.0,-0.061970,-0.210056,0.718936,196.0,-0.226961,0.703128,1,1,d0216r1p050200f0069
2006,d0216r1p050200c0131,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.086705,1.144509,1.187861,1.265896,1.132404,...,173.0,-0.033060,0.695204,-0.645200,173.0,0.692350,-0.677212,0,0,d0216r1p050200f0071
2007,d0216r1p050200c0134,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.111675,1.177665,1.213198,1.319797,1.173909,...,197.0,-0.054165,0.034340,0.774552,197.0,0.018615,0.763143,0,0,d0216r1p050200f0072


In [11]:
def reloc_yet_all(full_data, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy()
	working = working.sort_values(by = "ImageID")
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])
	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	#< Added NOV23
	df_all_rem["Yet"] = 0
	df_all_rem["Does"] = None
	df_all_rem["When"] = None

	df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].bfill()
	df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].bfill()

	working = df_all_rem.append(working)#. , ignore_index = True)

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [12]:
reloc_yet_all(test, "D:\ALL_FINAL\Combined_by_perc","TEST.parquet")

<ipython-input-11-4161d97405a7>:46: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,NaN,NaN
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,NaN,NaN
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,NaN,NaN
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,NaN,NaN
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,d0216r1p050200c0055,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.137755,1.193878,1.250000,1.345153,1.184821,...,196.0,-0.084049,-0.901397,0.718936,196.0,-0.911349,0.703128,1,1,d0216r1p050200f0034
1982,d0216r1p050200c0057,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.112821,1.168205,1.220513,1.289128,1.175799,...,195.0,-0.060177,-0.153889,0.663035,195.0,-0.170692,0.643113,1,1,d0216r1p050200f0036
1983,d0216r1p050200c0059,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.106383,1.159574,1.191489,1.310426,1.159505,...,188.0,-0.036109,0.599711,0.263514,188.0,0.594117,0.223010,0,0,d0216r1p050200f0038
1985,d0216r1p050200c0066,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.128065,1.209809,1.272207,1.387357,1.191644,...,183.5,-0.101659,-1.452802,-0.001256,183.5,-1.446480,-0.047057,1,1,d0216r1p050200f0043


In [13]:
def reloc_yet_all(full_data, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy()
	working = working.sort_values(by = "ImageID")
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])
	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	#< Added NOV23
	df_all_rem["Yet"] = 0
	df_all_rem["Does"] = None
	df_all_rem["When"] = None

	df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna('backfill')
	df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna('backfill')

	working = df_all_rem.append(working)#. , ignore_index = True)

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [14]:
reloc_yet_all(test, "D:\ALL_FINAL\Combined_by_perc","TEST.parquet")

<ipython-input-13-964aced605c3>:46: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



ArrowTypeError: ("Expected bytes, got a 'int' object", 'Conversion failed for column Does with type object')

In [15]:
full_data = test

In [16]:
working = full_data.copy()
working = working.sort_values(by = "ImageID")
df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

#>

working = working.loc[working["Frames_post_treatment"] >= 0]

# def complex_yet(x):
# 	ind = x["Relocalized"].idxmax()
# 	does = x.loc[ind]
# 	x["yet"] = 0
# 	x.loc[:ind, "yet"] = 0
# 	x.loc[ind:, "yet"] = does
# 	return

def reloc_yet(x):
	ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
	does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
	x.loc[:ind] = 0 #* Set all values less than the max value as 0
	x.loc[ind:] = does #* Set all values after the max value as 1
	return(x)

def workaround(ind):
	return(working.loc[ind, "Unique_Frame"])
def does_workaround(ind):
	return(working.loc[ind,"Relocalized"])

working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
working["When"] = pd.Series(working["ind"]).apply(workaround)
working.drop(columns='ind', inplace = True)

#< Added NOV23
df_all_rem["Yet"] = 0
df_all_rem["Does"] = None
df_all_rem["When"] = None

df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna('backfill')
df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna('backfill')

working = df_all_rem.append(working)#. , ignore_index = True)

working.to_parquet(os.path.join(post_path, output_file))

<ipython-input-16-7a2a8be28639>:43: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



NameError: name 'post_path' is not defined

In [17]:
working = full_data.copy()
working = working.sort_values(by = "ImageID")
df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

#>

working = working.loc[working["Frames_post_treatment"] >= 0]

# def complex_yet(x):
# 	ind = x["Relocalized"].idxmax()
# 	does = x.loc[ind]
# 	x["yet"] = 0
# 	x.loc[:ind, "yet"] = 0
# 	x.loc[ind:, "yet"] = does
# 	return

def reloc_yet(x):
	ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
	does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
	x.loc[:ind] = 0 #* Set all values less than the max value as 0
	x.loc[ind:] = does #* Set all values after the max value as 1
	return(x)

def workaround(ind):
	return(working.loc[ind, "Unique_Frame"])
def does_workaround(ind):
	return(working.loc[ind,"Relocalized"])

working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
working["When"] = pd.Series(working["ind"]).apply(workaround)
working.drop(columns='ind', inplace = True)

#< Added NOV23
df_all_rem["Yet"] = 0
df_all_rem["Does"] = None
df_all_rem["When"] = None

df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna('backfill')
df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna('backfill')

working = df_all_rem.append(working)#. , ignore_index = True)

<ipython-input-17-cdd9144b5bef>:43: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [18]:
working

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,backfill,backfill
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,backfill,backfill
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,backfill,backfill
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,backfill,backfill
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,backfill,backfill
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,d0216r1p050200c0055,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.137755,1.193878,1.250000,1.345153,1.184821,...,196.0,-0.084049,-0.901397,0.718936,196.0,-0.911349,0.703128,1,1,d0216r1p050200f0034
1982,d0216r1p050200c0057,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.112821,1.168205,1.220513,1.289128,1.175799,...,195.0,-0.060177,-0.153889,0.663035,195.0,-0.170692,0.643113,1,1,d0216r1p050200f0036
1983,d0216r1p050200c0059,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.106383,1.159574,1.191489,1.310426,1.159505,...,188.0,-0.036109,0.599711,0.263514,188.0,0.594117,0.223010,0,0,d0216r1p050200f0038
1985,d0216r1p050200c0066,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.128065,1.209809,1.272207,1.387357,1.191644,...,183.5,-0.101659,-1.452802,-0.001256,183.5,-1.446480,-0.047057,1,1,d0216r1p050200f0043


In [19]:
working = full_data.copy()
working = working.sort_values(by = "ImageID")
df_all_rem = working.loc[working["Frames_post_treatment"] < 0]

#>

working = working.loc[working["Frames_post_treatment"] >= 0]

# def complex_yet(x):
# 	ind = x["Relocalized"].idxmax()
# 	does = x.loc[ind]
# 	x["yet"] = 0
# 	x.loc[:ind, "yet"] = 0
# 	x.loc[ind:, "yet"] = does
# 	return

def reloc_yet(x):
	ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
	does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
	x.loc[:ind] = 0 #* Set all values less than the max value as 0
	x.loc[ind:] = does #* Set all values after the max value as 1
	return(x)

def workaround(ind):
	return(working.loc[ind, "Unique_Frame"])
def does_workaround(ind):
	return(working.loc[ind,"Relocalized"])

working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
working["When"] = pd.Series(working["ind"]).apply(workaround)
working.drop(columns='ind', inplace = True)

#< Added NOV23
df_all_rem["Yet"] = 0
df_all_rem["Does"] = None
df_all_rem["When"] = None

df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna(method = 'backfill')
df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna(method = 'backfill')

working = df_all_rem.append(working)#. , ignore_index = True)

<ipython-input-19-ad1a8c220843>:43: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [20]:
working

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,None,None
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,None,None
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,None,None
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,None,None
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,d0216r1p050200c0055,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.137755,1.193878,1.250000,1.345153,1.184821,...,196.0,-0.084049,-0.901397,0.718936,196.0,-0.911349,0.703128,1,1,d0216r1p050200f0034
1982,d0216r1p050200c0057,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.112821,1.168205,1.220513,1.289128,1.175799,...,195.0,-0.060177,-0.153889,0.663035,195.0,-0.170692,0.643113,1,1,d0216r1p050200f0036
1983,d0216r1p050200c0059,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.106383,1.159574,1.191489,1.310426,1.159505,...,188.0,-0.036109,0.599711,0.263514,188.0,0.594117,0.223010,0,0,d0216r1p050200f0038
1985,d0216r1p050200c0066,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.128065,1.209809,1.272207,1.387357,1.191644,...,183.5,-0.101659,-1.452802,-0.001256,183.5,-1.446480,-0.047057,1,1,d0216r1p050200f0043


In [21]:
working.set_index('Cell_Barcode')

,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,x95thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
Cell_Barcode,,,,,,,,,,,,,,,,,,,,,
d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,1.220337,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,None,None
d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,1.186448,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,None,None
d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,1.149793,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,None,None
d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,1.204835,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,None,None
d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,1.321327,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d0216r1p050200c0055,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.137755,1.193878,1.250000,1.345153,1.184821,1.240518,...,196.0,-0.084049,-0.901397,0.718936,196.0,-0.911349,0.703128,1,1,d0216r1p050200f0034
d0216r1p050200c0057,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.112821,1.168205,1.220513,1.289128,1.175799,1.228446,...,195.0,-0.060177,-0.153889,0.663035,195.0,-0.170692,0.643113,1,1,d0216r1p050200f0036
d0216r1p050200c0059,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.106383,1.159574,1.191489,1.310426,1.159505,1.191418,...,188.0,-0.036109,0.599711,0.263514,188.0,0.594117,0.223010,0,0,d0216r1p050200f0038


In [22]:
working.set_index(['Cell_Barcode', 'ImageID'])

,,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,x95thPercentile_norm_OBJ_Mean_Intensity_GFP,x99thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
Cell_Barcode,ImageID,,,,,,,,,,,,,,,,,,,,,
d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,1.220337,1.292789,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,None,None
d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,1.186448,1.274373,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,None,None
d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,1.149793,1.232903,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,None,None
d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,1.204835,1.326280,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,None,None
d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,1.321327,1.479611,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d0216r1p050200c0055,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.137755,1.193878,1.250000,1.345153,1.184821,1.240518,1.334949,...,196.0,-0.084049,-0.901397,0.718936,196.0,-0.911349,0.703128,1,1,d0216r1p050200f0034
d0216r1p050200c0057,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.112821,1.168205,1.220513,1.289128,1.175799,1.228446,1.297508,...,195.0,-0.060177,-0.153889,0.663035,195.0,-0.170692,0.643113,1,1,d0216r1p050200f0036
d0216r1p050200c0059,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.106383,1.159574,1.191489,1.310426,1.159505,1.191418,1.310347,...,188.0,-0.036109,0.599711,0.263514,188.0,0.594117,0.223010,0,0,d0216r1p050200f0038


In [23]:
os.getcwd(0)

TypeError: getcwd() takes no arguments (1 given)

In [24]:
os.getcwd()

"c:\\Users\\pcnba\\Grant Brown's Lab Dropbox\\Peter Bartlett\\Peter Bartlett Data\\Code\\single_cell_reloc\\single_cell_reloc_parquet\\Visualizations"

In [25]:
os.chdir("D:\ALL_FINAL\Combined_by_perc")

In [26]:
working.to_parquet('Testing.parquet')

In [27]:
working


,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578,0,None,None
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822,0,None,None
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578,0,None,None
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289,0,None,None
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1981,d0216r1p050200c0055,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.137755,1.193878,1.250000,1.345153,1.184821,...,196.0,-0.084049,-0.901397,0.718936,196.0,-0.911349,0.703128,1,1,d0216r1p050200f0034
1982,d0216r1p050200c0057,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.112821,1.168205,1.220513,1.289128,1.175799,...,195.0,-0.060177,-0.153889,0.663035,195.0,-0.170692,0.643113,1,1,d0216r1p050200f0036
1983,d0216r1p050200c0059,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.106383,1.159574,1.191489,1.310426,1.159505,...,188.0,-0.036109,0.599711,0.263514,188.0,0.594117,0.223010,0,0,d0216r1p050200f0038
1985,d0216r1p050200c0066,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.128065,1.209809,1.272207,1.387357,1.191644,...,183.5,-0.101659,-1.452802,-0.001256,183.5,-1.446480,-0.047057,1,1,d0216r1p050200f0043


In [28]:
from datetime import date
import os
import pandas as pd
import math
import os
import statistics
from joblib import Parallel, delayed
from scipy import stats
import plotly.express as px
import single_cell_reloc_parquet.global_functions.global_variables as gv
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotnine
plotly.io.kaleido.scope.mathjax = None #* This is an important line for timely svg output. This is to stop a call to the internet for prettier looking math
import kaleido
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context('paper')

print(plotly.__version__, kaleido.__version__)

# sns.set(rc = {'figure.figsize':(15,8)}) #* Set the figure size to be larger
# sns.set(rc = {'figure.figsize':(40,30)}) #* Set the figure size to be the shape of a powerpoint slide

5.13.0 0.2.1


In [29]:
test = pd.read_parquet("D:\ALL_FINAL\Combined_by_perc\ECO1_selected.parquet")

In [30]:
test

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,mKO_RAWfactor_lower,<lambda_0>_y,count,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund
0,d0222r1p240200c0001,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.051724,1.086207,1.112069,1.134310,1.092758,...,NaN,0.045370,60,116.0,0.010783,0.071360,-1.092248,116.0,0.047975,-1.003661
1,d0222r1p240200c0003,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.099476,1.162304,1.193717,1.259686,1.160967,...,NaN,0.045370,60,191.0,0.008672,0.023968,2.228172,191.0,0.001399,2.371881
2,d0222r1p240200c0004,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.075862,1.137931,1.185862,1.237034,1.140716,...,NaN,0.045370,60,145.0,-0.009473,-0.383432,0.393525,145.0,-0.394962,0.301548
3,d0222r1p240200c0008,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.079137,1.129496,1.152518,1.187914,1.118290,...,NaN,0.045370,60,139.0,-0.035391,-0.965332,0.112144,139.0,-0.948765,0.031505
4,d0222r1p240200c0009,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.060150,1.090977,1.135338,1.185113,1.090442,...,NaN,0.045370,60,133.0,-0.033030,-0.912331,-0.181656,133.0,-0.898916,-0.238538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6321,d0222r1p300200c0210,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.085161,1.141935,1.174194,1.232258,1.138366,...,NaN,0.021651,119,155.0,-0.017657,0.115568,2.209572,155.0,0.103095,2.183662
6322,d0222r1p300200c0220,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.076923,1.105983,1.129915,1.188376,1.099069,...,NaN,0.021651,119,117.0,-0.041007,-0.982078,-1.037841,117.0,-0.965302,-0.949117
6323,d0222r1p300200c0224,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.050847,1.078814,1.093220,1.111186,1.081057,...,NaN,0.021651,119,118.0,0.026152,2.174913,-0.939574,118.0,2.176137,-0.866675
6324,d0222r1p300200c0225,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.066116,1.090909,1.123967,1.177107,1.085932,...,NaN,0.021651,119,121.0,-0.031480,-0.534209,-0.649693,121.0,-0.532378,-0.619351


In [31]:
full_data

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,mKO_RAWfactor_lower,<lambda_0>_y,count,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund
0,d0216r1p050200c0002,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112500,1.171875,1.252734,1.327109,1.141569,...,192.099947,0.042893,9,128.0,-0.086234,-0.765151,-0.672143,128.0,-0.782225,-0.690578
1,d0216r1p050200c0010,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.100000,1.150000,1.200000,1.288929,1.137013,...,192.099947,0.042893,9,140.0,-0.043227,0.226201,0.781876,140.0,0.207931,0.771822
2,d0216r1p050200c0012,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.070312,1.112500,1.150000,1.233125,1.112300,...,192.099947,0.042893,9,128.0,-0.000667,1.207238,-0.672143,128.0,1.230613,-0.690578
3,d0216r1p050200c0014,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.097222,1.166667,1.218056,1.340833,1.154004,...,192.099947,0.042893,9,144.0,-0.058161,-0.118046,1.238968,144.0,-0.140737,1.259289
4,d0216r1p050200c0015,d0216r1p050200f0001,d0216,1,d0216r1p050200f0001,1.112676,1.204225,1.352113,1.514085,1.176807,...,192.099947,0.042893,9,142.0,-0.136160,-1.915984,1.012031,142.0,-1.879509,1.015556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004,d0216r1p050200c0123,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.105556,1.161111,1.188889,1.259833,1.159707,...,192.099947,0.031678,44,180.0,-0.033925,0.668127,-0.211716,180.0,0.664466,-0.257109
2005,d0216r1p050200c0127,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.130612,1.188776,1.222704,1.341786,1.182292,...,192.099947,0.031678,44,196.0,-0.061970,-0.210056,0.718936,196.0,-0.226961,0.703128
2006,d0216r1p050200c0131,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.086705,1.144509,1.187861,1.265896,1.132404,...,192.099947,0.031678,44,173.0,-0.033060,0.695204,-0.645200,173.0,0.692350,-0.677212
2007,d0216r1p050200c0134,d0216r1p050200f0084,d0216,84,d0216r1p050200f0084,1.111675,1.177665,1.213198,1.319797,1.173909,...,192.099947,0.031678,44,197.0,-0.054165,0.034340,0.774552,197.0,0.018615,0.763143


In [32]:
full_data.Protein.unique()

array(['ZPR1'], dtype=object)

In [33]:
full_data = test

In [34]:
test

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,mKO_RAWfactor_lower,<lambda_0>_y,count,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund
0,d0222r1p240200c0001,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.051724,1.086207,1.112069,1.134310,1.092758,...,NaN,0.045370,60,116.0,0.010783,0.071360,-1.092248,116.0,0.047975,-1.003661
1,d0222r1p240200c0003,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.099476,1.162304,1.193717,1.259686,1.160967,...,NaN,0.045370,60,191.0,0.008672,0.023968,2.228172,191.0,0.001399,2.371881
2,d0222r1p240200c0004,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.075862,1.137931,1.185862,1.237034,1.140716,...,NaN,0.045370,60,145.0,-0.009473,-0.383432,0.393525,145.0,-0.394962,0.301548
3,d0222r1p240200c0008,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.079137,1.129496,1.152518,1.187914,1.118290,...,NaN,0.045370,60,139.0,-0.035391,-0.965332,0.112144,139.0,-0.948765,0.031505
4,d0222r1p240200c0009,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.060150,1.090977,1.135338,1.185113,1.090442,...,NaN,0.045370,60,133.0,-0.033030,-0.912331,-0.181656,133.0,-0.898916,-0.238538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6321,d0222r1p300200c0210,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.085161,1.141935,1.174194,1.232258,1.138366,...,NaN,0.021651,119,155.0,-0.017657,0.115568,2.209572,155.0,0.103095,2.183662
6322,d0222r1p300200c0220,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.076923,1.105983,1.129915,1.188376,1.099069,...,NaN,0.021651,119,117.0,-0.041007,-0.982078,-1.037841,117.0,-0.965302,-0.949117
6323,d0222r1p300200c0224,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.050847,1.078814,1.093220,1.111186,1.081057,...,NaN,0.021651,119,118.0,0.026152,2.174913,-0.939574,118.0,2.176137,-0.866675
6324,d0222r1p300200c0225,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.066116,1.090909,1.123967,1.177107,1.085932,...,NaN,0.021651,119,121.0,-0.031480,-0.534209,-0.649693,121.0,-0.532378,-0.619351


In [35]:
test.protein.unique()

AttributeError: 'DataFrame' object has no attribute 'protein'

In [36]:
test.Protein.unique()

array(['ECO1'], dtype=object)

In [37]:
full_data = test

In [38]:
def reloc_yet_all(full_data, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy()
	working = working.sort_values(by = "ImageID")
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]
	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])
	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	#< Added NOV23
	df_all_rem["Yet"] = 0
	df_all_rem["Does"] = None
	df_all_rem["When"] = None

	df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna(method = 'backfill')
	df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna(method = 'backfill')

	working = df_all_rem.append(working)#. , ignore_index = True)

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [39]:
test = pd.read_parquet("D:\ALL_FINAL\Combined_by_perc\ECO1_selected.parquet")

In [40]:
working = test.copy() #. full_data.copy()
working = working.sort_values(by = "ImageID")
df_all_rem = working.loc[working["Frames_post_treatment"] < 0]
#>

working = working.loc[working["Frames_post_treatment"] >= 0]

# def complex_yet(x):
# 	ind = x["Relocalized"].idxmax()
# 	does = x.loc[ind]
# 	x["yet"] = 0
# 	x.loc[:ind, "yet"] = 0
# 	x.loc[ind:, "yet"] = does
# 	return

def reloc_yet(x):
	ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
	does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
	x.loc[:ind] = 0 #* Set all values less than the max value as 0
	x.loc[ind:] = does #* Set all values after the max value as 1
	return(x)

def workaround(ind):
	return(working.loc[ind, "Unique_Frame"])
def does_workaround(ind):
	return(working.loc[ind,"Relocalized"])

working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
working["When"] = pd.Series(working["ind"]).apply(workaround)
working.drop(columns='ind', inplace = True)

#< Added NOV23
df_all_rem["Yet"] = 0
df_all_rem["Does"] = None
df_all_rem["When"] = None

df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna(method = 'backfill')
df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna(method = 'backfill')

working = df_all_rem.append(working)#. , ignore_index = True)

<ipython-input-40-adef9efec9ba>:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [41]:
working

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
0,d0222r1p240200c0001,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.051724,1.086207,1.112069,1.134310,1.092758,...,116.0,0.010783,0.071360,-1.092248,116.0,0.047975,-1.003661,0,None,None
24,d0222r1p240200c0057,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.060345,1.094828,1.137931,1.211724,1.085761,...,116.0,-0.030146,-0.847576,-1.092248,116.0,-0.837851,-1.003661,0,None,None
22,d0222r1p240200c0042,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.069421,1.102479,1.138017,1.327190,1.092876,...,121.0,0.060874,1.195999,-0.811262,121.0,1.182609,-0.778625,0,None,None
21,d0222r1p240200c0040,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.136364,1.233766,1.275974,1.337662,1.211293,...,154.0,0.068733,1.372462,0.794485,154.0,1.365856,0.706614,0,None,None
20,d0222r1p240200c0039,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.120879,1.170330,1.229945,1.313187,1.168059,...,182.0,0.050267,0.957848,1.906794,182.0,0.937575,1.966816,0,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,d0222r1p300200c0049,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.073770,1.114754,1.147541,1.233443,1.107111,...,122.0,-0.016696,0.160734,-0.554661,122.0,0.147595,-0.536909,1,1,d0222r1p300200f0027
6309,d0222r1p300200c0038,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.077922,1.129870,1.168831,1.251234,1.126831,...,154.0,-0.002375,0.833924,2.134839,154.0,0.815938,2.101220,1,1,d0222r1p300200f0027
6308,d0222r1p300200c0012,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.090278,1.131944,1.177778,1.248889,1.128503,...,144.0,-0.004251,0.745747,1.359627,144.0,0.727850,1.276805,1,1,d0222r1p300200f0029
6315,d0222r1p300200c0145,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.087248,1.139597,1.174497,1.285638,1.133359,...,149.0,0.024749,2.108986,1.753738,149.0,2.108355,1.689013,1,1,d0222r1p300200f0037


In [42]:
new_try = working.query('Frames_post_treatment' >= 0)

TypeError: '>=' not supported between instances of 'str' and 'int'

In [43]:
working.Frames_post_treatment.type()

AttributeError: 'Series' object has no attribute 'type'

In [44]:
working.Frames_post_treatment.dtype()

TypeError: 'numpy.dtype[int64]' object is not callable

In [45]:
dtype(working.Frames_post_treatment)

NameError: name 'dtype' is not defined

In [46]:
test

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,mKO_RAWfactor_lower,<lambda_0>_y,count,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund
0,d0222r1p240200c0001,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.051724,1.086207,1.112069,1.134310,1.092758,...,NaN,0.045370,60,116.0,0.010783,0.071360,-1.092248,116.0,0.047975,-1.003661
1,d0222r1p240200c0003,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.099476,1.162304,1.193717,1.259686,1.160967,...,NaN,0.045370,60,191.0,0.008672,0.023968,2.228172,191.0,0.001399,2.371881
2,d0222r1p240200c0004,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.075862,1.137931,1.185862,1.237034,1.140716,...,NaN,0.045370,60,145.0,-0.009473,-0.383432,0.393525,145.0,-0.394962,0.301548
3,d0222r1p240200c0008,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.079137,1.129496,1.152518,1.187914,1.118290,...,NaN,0.045370,60,139.0,-0.035391,-0.965332,0.112144,139.0,-0.948765,0.031505
4,d0222r1p240200c0009,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.060150,1.090977,1.135338,1.185113,1.090442,...,NaN,0.045370,60,133.0,-0.033030,-0.912331,-0.181656,133.0,-0.898916,-0.238538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6321,d0222r1p300200c0210,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.085161,1.141935,1.174194,1.232258,1.138366,...,NaN,0.021651,119,155.0,-0.017657,0.115568,2.209572,155.0,0.103095,2.183662
6322,d0222r1p300200c0220,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.076923,1.105983,1.129915,1.188376,1.099069,...,NaN,0.021651,119,117.0,-0.041007,-0.982078,-1.037841,117.0,-0.965302,-0.949117
6323,d0222r1p300200c0224,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.050847,1.078814,1.093220,1.111186,1.081057,...,NaN,0.021651,119,118.0,0.026152,2.174913,-0.939574,118.0,2.176137,-0.866675
6324,d0222r1p300200c0225,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.066116,1.090909,1.123967,1.177107,1.085932,...,NaN,0.021651,119,121.0,-0.031480,-0.534209,-0.649693,121.0,-0.532378,-0.619351


In [47]:
def reloc_yet_all(full_data, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy() #. full_data.copy()
	working = working.sort_values(by = "ImageID")
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]
	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])
	
	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	working["Yes_yet"] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 1].count())
	working['No_yet'] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 0].count())

	#< Added NOV23
	# df_all_rem["Yet"] = 0
	# df_all_rem["Does"] = None
	# df_all_rem["When"] = None

	# df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna(method = 'backfill')
	# df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna(method = 'backfill')
	#>

	# working = df_all_rem.append(working)#. , ignore_index = True)
	# for now have just values after treatment

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [48]:
reloc_yet_all(test)

TypeError: reloc_yet_all() missing 2 required positional arguments: 'post_path' and 'output_file'

In [49]:
def reloc_yet_all(full_data):#, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy() #. full_data.copy()
	working = working.sort_values(by = "ImageID")
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]
	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])

	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	working["Yes_yet"] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 1].count())
	working['No_yet'] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 0].count())

	#< Added NOV23
	# df_all_rem["Yet"] = 0
	# df_all_rem["Does"] = None
	# df_all_rem["When"] = None

	# df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna(method = 'backfill')
	# df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna(method = 'backfill')
	#>

	# working = df_all_rem.append(working)#. , ignore_index = True)
	# for now have just values after treatment

	working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [50]:
reloc_yet_all(test)

NameError: name 'post_path' is not defined

In [51]:
def reloc_yet_all(full_data):#, post_path, output_file):
	#<NOV23' ADD
	working = full_data.copy() #. full_data.copy()
	working = working.sort_values(by = "ImageID")
	df_all_rem = working.loc[working["Frames_post_treatment"] < 0]
	#>

	working = working.loc[working["Frames_post_treatment"] >= 0]

	# def complex_yet(x):
	# 	ind = x["Relocalized"].idxmax()
	# 	does = x.loc[ind]
	# 	x["yet"] = 0
	# 	x.loc[:ind, "yet"] = 0
	# 	x.loc[ind:, "yet"] = does
	# 	return

	def reloc_yet(x):
		ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
		does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
		x.loc[:ind] = 0 #* Set all values less than the max value as 0
		x.loc[ind:] = does #* Set all values after the max value as 1
		return(x)

	def workaround(ind):
		return(working.loc[ind, "Unique_Frame"])

	def does_workaround(ind):
		return(working.loc[ind,"Relocalized"])

	working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
	working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
	working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
	working["When"] = pd.Series(working["ind"]).apply(workaround)
	working.drop(columns='ind', inplace = True)

	working["Yes_yet"] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 1].count())
	working['No_yet'] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 0].count())

	#< Added NOV23
	# df_all_rem["Yet"] = 0
	# df_all_rem["Does"] = None
	# df_all_rem["When"] = None

	# df_all_rem["Does"] = df_all_rem.groupby(by = "Cell_Barcode")["Does"].fillna(method = 'backfill')
	# df_all_rem["When"] = df_all_rem.groupby(by = "Cell_Barcode")["When"].fillna(method = 'backfill')
	#>

	# working = df_all_rem.append(working)#. , ignore_index = True)
	# for now have just values after treatment

	# working.to_parquet(os.path.join(post_path, output_file))
	return(working)
	#>

In [52]:
new = reloc_yet_all(test)

In [53]:
new

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When,Yes_yet,No_yet
1757,d0222r1p240200c0010,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.058333,1.091667,1.125000,1.180667,1.086707,...,-0.449947,-0.704338,120.0,-0.451634,-0.698809,0,1,d0222r1p240200f0034,NaN,NaN
1763,d0222r1p240200c0026,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.102190,1.160584,1.210219,1.291971,1.149427,...,2.406971,1.269937,137.0,2.422100,1.249471,1,1,d0222r1p240200f0027,NaN,NaN
1762,d0222r1p240200c0023,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.093243,1.155405,1.183784,1.227297,1.158319,...,0.087363,2.420792,148.0,0.075054,2.510124,0,1,d0222r1p240200f0031,NaN,NaN
1761,d0222r1p240200c0020,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.066116,1.105785,1.132231,1.198347,1.099080,...,-0.990868,-0.580674,121.0,-0.975569,-0.584205,0,0,d0222r1p240200f0027,NaN,NaN
1760,d0222r1p240200c0019,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.066667,1.108333,1.141667,1.241667,1.098847,...,0.613132,-0.704338,120.0,0.596528,-0.698809,0,1,d0222r1p240200f0028,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,d0222r1p300200c0049,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.073770,1.114754,1.147541,1.233443,1.107111,...,0.160734,-0.554661,122.0,0.147595,-0.536909,1,1,d0222r1p300200f0027,NaN,NaN
6309,d0222r1p300200c0038,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.077922,1.129870,1.168831,1.251234,1.126831,...,0.833924,2.134839,154.0,0.815938,2.101220,1,1,d0222r1p300200f0027,NaN,NaN
6308,d0222r1p300200c0012,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.090278,1.131944,1.177778,1.248889,1.128503,...,0.745747,1.359627,144.0,0.727850,1.276805,1,1,d0222r1p300200f0029,NaN,NaN
6315,d0222r1p300200c0145,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.087248,1.139597,1.174497,1.285638,1.133359,...,2.108986,1.753738,149.0,2.108355,1.689013,1,1,d0222r1p300200f0037,NaN,NaN


In [54]:
test

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,mKO_RAWfactor_lower,<lambda_0>_y,count,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund
0,d0222r1p240200c0001,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.051724,1.086207,1.112069,1.134310,1.092758,...,NaN,0.045370,60,116.0,0.010783,0.071360,-1.092248,116.0,0.047975,-1.003661
1,d0222r1p240200c0003,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.099476,1.162304,1.193717,1.259686,1.160967,...,NaN,0.045370,60,191.0,0.008672,0.023968,2.228172,191.0,0.001399,2.371881
2,d0222r1p240200c0004,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.075862,1.137931,1.185862,1.237034,1.140716,...,NaN,0.045370,60,145.0,-0.009473,-0.383432,0.393525,145.0,-0.394962,0.301548
3,d0222r1p240200c0008,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.079137,1.129496,1.152518,1.187914,1.118290,...,NaN,0.045370,60,139.0,-0.035391,-0.965332,0.112144,139.0,-0.948765,0.031505
4,d0222r1p240200c0009,d0222r1p240200f0001,d0222,1,d0222r1p240200f0001,1.060150,1.090977,1.135338,1.185113,1.090442,...,NaN,0.045370,60,133.0,-0.033030,-0.912331,-0.181656,133.0,-0.898916,-0.238538
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6321,d0222r1p300200c0210,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.085161,1.141935,1.174194,1.232258,1.138366,...,NaN,0.021651,119,155.0,-0.017657,0.115568,2.209572,155.0,0.103095,2.183662
6322,d0222r1p300200c0220,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.076923,1.105983,1.129915,1.188376,1.099069,...,NaN,0.021651,119,117.0,-0.041007,-0.982078,-1.037841,117.0,-0.965302,-0.949117
6323,d0222r1p300200c0224,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.050847,1.078814,1.093220,1.111186,1.081057,...,NaN,0.021651,119,118.0,0.026152,2.174913,-0.939574,118.0,2.176137,-0.866675
6324,d0222r1p300200c0225,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.066116,1.090909,1.123967,1.177107,1.085932,...,NaN,0.021651,119,121.0,-0.031480,-0.534209,-0.649693,121.0,-0.532378,-0.619351


In [55]:
working = test.copy() #. full_data.copy()
working = working.sort_values(by = "ImageID")
df_all_rem = working.loc[working["Frames_post_treatment"] < 0]
#>

working = working.loc[working["Frames_post_treatment"] >= 0]

# def complex_yet(x):
# 	ind = x["Relocalized"].idxmax()
# 	does = x.loc[ind]
# 	x["yet"] = 0
# 	x.loc[:ind, "yet"] = 0
# 	x.loc[ind:, "yet"] = does
# 	return

def reloc_yet(x):
	ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
	does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
	x.loc[:ind] = 0 #* Set all values less than the max value as 0
	x.loc[ind:] = does #* Set all values after the max value as 1
	return(x)

def workaround(ind):
	return(working.loc[ind, "Unique_Frame"])

def does_workaround(ind):
	return(working.loc[ind,"Relocalized"])

working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
working["When"] = pd.Series(working["ind"]).apply(workaround)
working.drop(columns='ind', inplace = True)

In [56]:
working

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Abundance,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When
1757,d0222r1p240200c0010,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.058333,1.091667,1.125000,1.180667,1.086707,...,120.0,-0.029272,-0.449947,-0.704338,120.0,-0.451634,-0.698809,0,1,d0222r1p240200f0034
1763,d0222r1p240200c0026,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.102190,1.160584,1.210219,1.291971,1.149427,...,137.0,0.033979,2.406971,1.269937,137.0,2.422100,1.249471,1,1,d0222r1p240200f0027
1762,d0222r1p240200c0023,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.093243,1.155405,1.183784,1.227297,1.158319,...,148.0,-0.017376,0.087363,2.420792,148.0,0.075054,2.510124,0,1,d0222r1p240200f0031
1761,d0222r1p240200c0020,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.066116,1.105785,1.132231,1.198347,1.099080,...,121.0,-0.041247,-0.990868,-0.580674,121.0,-0.975569,-0.584205,0,0,d0222r1p240200f0027
1760,d0222r1p240200c0019,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.066667,1.108333,1.141667,1.241667,1.098847,...,120.0,-0.005736,0.613132,-0.704338,120.0,0.596528,-0.698809,0,1,d0222r1p240200f0028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,d0222r1p300200c0049,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.073770,1.114754,1.147541,1.233443,1.107111,...,122.0,-0.016696,0.160734,-0.554661,122.0,0.147595,-0.536909,1,1,d0222r1p300200f0027
6309,d0222r1p300200c0038,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.077922,1.129870,1.168831,1.251234,1.126831,...,154.0,-0.002375,0.833924,2.134839,154.0,0.815938,2.101220,1,1,d0222r1p300200f0027
6308,d0222r1p300200c0012,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.090278,1.131944,1.177778,1.248889,1.128503,...,144.0,-0.004251,0.745747,1.359627,144.0,0.727850,1.276805,1,1,d0222r1p300200f0029
6315,d0222r1p300200c0145,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.087248,1.139597,1.174497,1.285638,1.133359,...,149.0,0.024749,2.108986,1.753738,149.0,2.108355,1.689013,1,1,d0222r1p300200f0037


In [57]:
working["Yes_yet"] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 1].count())

In [58]:
working

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,log_Loc_score,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When,Yes_yet
1757,d0222r1p240200c0010,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.058333,1.091667,1.125000,1.180667,1.086707,...,-0.029272,-0.449947,-0.704338,120.0,-0.451634,-0.698809,0,1,d0222r1p240200f0034,NaN
1763,d0222r1p240200c0026,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.102190,1.160584,1.210219,1.291971,1.149427,...,0.033979,2.406971,1.269937,137.0,2.422100,1.249471,1,1,d0222r1p240200f0027,NaN
1762,d0222r1p240200c0023,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.093243,1.155405,1.183784,1.227297,1.158319,...,-0.017376,0.087363,2.420792,148.0,0.075054,2.510124,0,1,d0222r1p240200f0031,NaN
1761,d0222r1p240200c0020,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.066116,1.105785,1.132231,1.198347,1.099080,...,-0.041247,-0.990868,-0.580674,121.0,-0.975569,-0.584205,0,0,d0222r1p240200f0027,NaN
1760,d0222r1p240200c0019,d0222r1p240200f0027,d0222,27,d0222r1p240200f0027,1.066667,1.108333,1.141667,1.241667,1.098847,...,-0.005736,0.613132,-0.704338,120.0,0.596528,-0.698809,0,1,d0222r1p240200f0028,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6310,d0222r1p300200c0049,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.073770,1.114754,1.147541,1.233443,1.107111,...,-0.016696,0.160734,-0.554661,122.0,0.147595,-0.536909,1,1,d0222r1p300200f0027,NaN
6309,d0222r1p300200c0038,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.077922,1.129870,1.168831,1.251234,1.126831,...,-0.002375,0.833924,2.134839,154.0,0.815938,2.101220,1,1,d0222r1p300200f0027,NaN
6308,d0222r1p300200c0012,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.090278,1.131944,1.177778,1.248889,1.128503,...,-0.004251,0.745747,1.359627,144.0,0.727850,1.276805,1,1,d0222r1p300200f0029,NaN
6315,d0222r1p300200c0145,d0222r1p300200f0069,d0222,69,d0222r1p300200f0069,1.087248,1.139597,1.174497,1.285638,1.133359,...,0.024749,2.108986,1.753738,149.0,2.108355,1.689013,1,1,d0222r1p300200f0037,NaN


In [59]:
working["Yes_yet"] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 1].size())

TypeError: 'int' object is not callable

In [60]:
working.dtypes()

TypeError: 'Series' object is not callable

In [61]:
working.dtypes()

TypeError: 'Series' object is not callable

In [62]:
working.dtypes

Cell_Barcode      object
ImageID           object
Date              object
Frame              int64
Unique_Frame      object
                  ...   
z_score_Abund    float64
Yet                int64
Does               int64
When              object
Yes_yet          float64
Length: 119, dtype: object

In [63]:
working.set_index('Unique_Frame').Yet.eq(1).sum(level=0).astype(int).reset_index()

<ipython-input-63-cb35039f553e>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


,Unique_Frame,Yet
0,d0222r1p240200f0027,6
1,d0222r1p240200f0028,8
2,d0222r1p240200f0029,10
3,d0222r1p240200f0030,12
4,d0222r1p240200f0031,13
...,...,...
124,d0222r1p300200f0065,14
125,d0222r1p300200f0066,14
126,d0222r1p300200f0067,14
127,d0222r1p300200f0068,14


In [64]:
working["Yes_yet"] = working.groupby('Unique_Frame')['Yet'].apply(lambda x: x[x == 0].size())

TypeError: 'int' object is not callable

In [65]:
working.set_index('Unique_Frame').Yet.eq(0).sum(level=0).astype(int).reset_index()

<ipython-input-65-9d5a7f7f013b>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


,Unique_Frame,Yet
0,d0222r1p240200f0027,26
1,d0222r1p240200f0028,24
2,d0222r1p240200f0029,22
3,d0222r1p240200f0030,21
4,d0222r1p240200f0031,20
...,...,...
124,d0222r1p300200f0065,4
125,d0222r1p300200f0066,4
126,d0222r1p300200f0067,4
127,d0222r1p300200f0068,4


In [66]:
test = pd.read_parquet("D:\ALL_FINAL\Combined_by_perc\FLR1_selected.parquet")

In [67]:
working = test.copy() #. full_data.copy()
working = working.sort_values(by = "ImageID")
df_all_rem = working.loc[working["Frames_post_treatment"] < 0]
#>
working = working.loc[working["Frames_post_treatment"] >= 0]

# def complex_yet(x):
# 	ind = x["Relocalized"].idxmax()
# 	does = x.loc[ind]
# 	x["yet"] = 0
# 	x.loc[:ind, "yet"] = 0
# 	x.loc[ind:, "yet"] = does
# 	return

def reloc_yet(x):
	ind = x.idxmax() #* Get the first occurrence of max value. This is 1 when relocalised, so the first time in the 'Relocalized' series where true
	does = x.loc[ind] #* Get the value of max value. ie. If max value is 0, then it never relocalizes, whereas if 1 then it does at some point
	x.loc[:ind] = 0 #* Set all values less than the max value as 0
	x.loc[ind:] = does #* Set all values after the max value as 1
	return(x)

def workaround(ind):
	return(working.loc[ind, "Unique_Frame"])

def does_workaround(ind):
	return(working.loc[ind,"Relocalized"])

working["Yet"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform(reloc_yet) # This repesents wether there has been relocaliztion yet
working["ind"] = working.groupby(by = "Cell_Barcode")["Relocalized"].transform('idxmax')
working["Does"] = pd.Series(working["ind"]).apply(does_workaround) #this will work for now but should make it come out of one of the other functions
working["When"] = pd.Series(working["ind"]).apply(workaround)
working.drop(columns='ind', inplace = True)


In [68]:
working.Protein.unique()

array(['FLR1'], dtype=object)

In [69]:
working.set_index('Unique_Frame').Yet.eq(0).sum(level=0).astype(int).reset_index()

<ipython-input-69-9d5a7f7f013b>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


,Unique_Frame,Yet
0,d0222r2p240200f0027,26
1,d0222r2p240200f0028,26
2,d0222r2p240200f0029,26
3,d0222r2p240200f0030,24
4,d0222r2p240200f0031,19
...,...,...
124,d0222r2p300200f0065,1
125,d0222r2p300200f0066,1
126,d0222r2p300200f0067,1
127,d0222r2p300200f0068,1


In [70]:
info = working.Protein.unique()

In [71]:
info_yes.set_index("Unique_Frame")

NameError: name 'info_yes' is not defined

In [72]:
info_yes = working.set_index('Unique_Frame').Yet.eq(0).sum(level=0).astype(int).reset_index()
info_yes.set_index("Unique_Frame")

<ipython-input-72-db5d1e16d933>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


,Yet
Unique_Frame,
d0222r2p240200f0027,26
d0222r2p240200f0028,26
d0222r2p240200f0029,26
d0222r2p240200f0030,24
d0222r2p240200f0031,19
...,...
d0222r2p300200f0065,1
d0222r2p300200f0066,1
d0222r2p300200f0067,1


In [73]:
info_yes = working.set_index('Unique_Frame').Yet.eq(1).sum(level=0).astype(int).reset_index()
info_yes.set_index("Unique_Frame", inplace = True).rename(columns={"Yet": "Yes_yet")


SyntaxError: closing parenthesis ')' does not match opening parenthesis '{' (<ipython-input-73-3ebbedca9ee9>, line 2)

In [74]:
info_yes = working.set_index('Unique_Frame').Yet.eq(1).sum(level=0).astype(int).reset_index()
info_yes.set_index("Unique_Frame", inplace = True).rename(columns={"Yet": "Yes_yet"})

info_no = working.set_index('Unique_Frame').Yet.eq(0).sum(level=0).astype(int).reset_index()

info_no.set_index("Unique_Frame", inplace = True).rename(columns={"Yet": "No_yet"})

newest = pd.merge(working, info_yes, left_on = "Unique_Frame", right_index = True)
newest = pd.merge(newest, info_no, left_on = "Unique_Frame", right_index = True)

<ipython-input-74-d0ff37dc3139>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


AttributeError: 'NoneType' object has no attribute 'rename'

In [75]:
type(info_yes)

pandas.core.frame.DataFrame

In [76]:
info_yes = working.set_index('Unique_Frame').Yet.eq(1).sum(level=0).astype(int).reset_index()
info_yes.set_index("Unique_Frame", inplace = True)
info_yes.rename(columns={"Yet": "Yes_yet"}, inplace = True)

<ipython-input-76-d9595f9d5197>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


In [77]:
info_yes

,Yes_yet
Unique_Frame,
d0222r2p240200f0027,12
d0222r2p240200f0028,14
d0222r2p240200f0029,16
d0222r2p240200f0030,18
d0222r2p240200f0031,25
...,...
d0222r2p300200f0065,124
d0222r2p300200f0066,124
d0222r2p300200f0067,125


In [78]:
info_yes = working.set_index('Unique_Frame').Yet.eq(1).sum(level=0).astype(int).reset_index()
info_yes.set_index("Unique_Frame", inplace = True)
info_yes.rename(columns={"Yet": "Yes_yet"}, inplace = True)


info_no = working.set_index('Unique_Frame').Yet.eq(0).sum(level=0).astype(int).reset_index()
info_no.set_index('Unique_Frame', inplace = True)
info_no.rename(columns={"Yet": "No_yet"}, inplace = True)

newest = pd.merge(working, info_yes, left_on = "Unique_Frame", right_index = True)
newest = pd.merge(newest, info_no, left_on = "Unique_Frame", right_index = True)

<ipython-input-78-94943a258cce>:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
<ipython-input-78-94943a258cce>:6: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().


In [79]:
newest

,Cell_Barcode,ImageID,Date,Frame,Unique_Frame,x80thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Median_GFP,x95thPercentile_norm_OBJ_Median_GFP,x99thPercentile_norm_OBJ_Median_GFP,x90thPercentile_norm_OBJ_Mean_Intensity_GFP,...,z_score_logLoc,z_score_logAbund,Abundance,z_score_Loc,z_score_Abund,Yet,Does,When,Yes_yet,No_yet
3075,d0222r2p240200c0217,d0222r2p240200f0027,d0222,27,d0222r2p240200f0027,1.069930,1.097902,1.126224,1.205664,1.100529,...,-0.586577,0.332622,143.0,-0.577398,0.265058,0,1,d0222r2p240200f0028,12,26
3069,d0222r2p240200c0195,d0222r2p240200f0027,d0222,27,d0222r2p240200f0027,1.059259,1.096296,1.133333,1.177778,1.094062,...,-0.596699,-0.284848,135.0,-0.587040,-0.303279,0,1,d0222r2p240200f0033,12,26
3070,d0222r2p240200c0201,d0222r2p240200f0027,d0222,27,d0222r2p240200f0027,1.074627,1.122388,1.170149,1.241791,1.120679,...,0.536345,2.028754,167.5,0.509464,2.005590,1,1,d0222r2p240200f0027,12,26
3071,d0222r2p240200c0203,d0222r2p240200f0027,d0222,27,d0222r2p240200f0027,1.061538,1.093846,1.120000,1.176923,1.096757,...,-0.390168,-0.689636,130.0,-0.389773,-0.658490,0,1,d0222r2p240200f0036,12,26
3072,d0222r2p240200c0204,d0222r2p240200f0027,d0222,27,d0222r2p240200f0027,1.080247,1.129630,1.160494,1.226914,1.126521,...,0.242677,1.670659,162.0,0.221892,1.614858,1,1,d0222r2p240200f0027,12,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15808,d0222r2p300200c0319,d0222r2p300200f0069,d0222,69,d0222r2p300200f0069,1.112500,1.193750,1.262500,1.367375,1.173137,...,-0.127916,-1.384563,160.0,-0.159240,-1.266937,1,1,d0222r2p300200f0038,125,2
15807,d0222r2p300200c0317,d0222r2p300200f0069,d0222,69,d0222r2p300200f0069,1.072368,1.111842,1.144737,1.203947,1.112004,...,-1.117988,-1.567800,152.0,-0.980147,-1.386267,1,1,d0222r2p300200f0038,125,2
15806,d0222r2p300200c0313,d0222r2p300200f0069,d0222,69,d0222r2p300200f0069,1.073446,1.129944,1.158192,1.241808,1.127587,...,-0.999835,-1.023843,177.0,-0.886352,-1.013362,1,1,d0222r2p300200f0049,125,2
15836,d0222r2p300200c0496,d0222r2p300200f0069,d0222,69,d0222r2p300200f0069,1.108696,1.168478,1.219565,1.325217,1.150065,...,-0.477755,-0.885286,184.0,-0.458531,-0.908949,1,1,d0222r2p300200f0038,125,2


In [80]:
newest.Protein.unique()

array(['FLR1'], dtype=object)

In [81]:
newest[['ImageID', 'Frames_post_treatment', 'Yes_yet', 'No_yet']].unique()

AttributeError: 'DataFrame' object has no attribute 'unique'

In [82]:
lview = newest[['ImageID', 'Frames_post_treatment', 'Yes_yet', 'No_yet']].drop_duplicates()

In [83]:
view

NameError: name 'view' is not defined

In [84]:
lview

,ImageID,Frames_post_treatment,Yes_yet,No_yet
3075,d0222r2p240200f0027,0,12,26
3244,d0222r2p240200f0028,1,14,26
3424,d0222r2p240200f0029,2,16,26
3596,d0222r2p240200f0030,3,18,24
3785,d0222r2p240200f0031,4,25,19
...,...,...,...,...
14258,d0222r2p300200f0065,38,124,1
14682,d0222r2p300200f0066,39,124,1
15026,d0222r2p300200f0067,40,125,1
15397,d0222r2p300200f0068,41,125,1


In [85]:
lview = newest[['ImageID', 'Frames_post_treatment', 'Yes_yet', 'No_yet', 'Time_post_treatment']].drop_duplicates()

KeyError: "['Time_post_treatment'] not in index"

In [86]:
newest.columns

Index(['Cell_Barcode', 'ImageID', 'Date', 'Frame', 'Unique_Frame',
       'x80thPercentile_norm_OBJ_Median_GFP',
       'x90thPercentile_norm_OBJ_Median_GFP',
       'x95thPercentile_norm_OBJ_Median_GFP',
       'x99thPercentile_norm_OBJ_Median_GFP',
       'x90thPercentile_norm_OBJ_Mean_Intensity_GFP',
       ...
       'z_score_logLoc', 'z_score_logAbund', 'Abundance', 'z_score_Loc',
       'z_score_Abund', 'Yet', 'Does', 'When', 'Yes_yet', 'No_yet'],
      dtype='object', length=120)

In [87]:
lview['Time_post'] = lview['Frames_post_treatment']*7.5

In [88]:
lview

,ImageID,Frames_post_treatment,Yes_yet,No_yet,Time_post
3075,d0222r2p240200f0027,0,12,26,0.0
3244,d0222r2p240200f0028,1,14,26,7.5
3424,d0222r2p240200f0029,2,16,26,15.0
3596,d0222r2p240200f0030,3,18,24,22.5
3785,d0222r2p240200f0031,4,25,19,30.0
...,...,...,...,...,...
14258,d0222r2p300200f0065,38,124,1,285.0
14682,d0222r2p300200f0066,39,124,1,292.5
15026,d0222r2p300200f0067,40,125,1,300.0
15397,d0222r2p300200f0068,41,125,1,307.5


In [89]:
lview.set_index('Time_post')

,ImageID,Frames_post_treatment,Yes_yet,No_yet
Time_post,,,,
0.0,d0222r2p240200f0027,0,12,26
7.5,d0222r2p240200f0028,1,14,26
15.0,d0222r2p240200f0029,2,16,26
22.5,d0222r2p240200f0030,3,18,24
30.0,d0222r2p240200f0031,4,25,19
...,...,...,...,...
285.0,d0222r2p300200f0065,38,124,1
292.5,d0222r2p300200f0066,39,124,1
300.0,d0222r2p300200f0067,40,125,1


In [90]:
lview.loc[0]

KeyError: 0

In [91]:
lview.loc['0']

KeyError: '0'

In [92]:
lview.loc[0.0]

KeyError: 0.0

In [93]:
lview.loc['0.0']

KeyError: '0.0'

In [94]:
lview


,ImageID,Frames_post_treatment,Yes_yet,No_yet,Time_post
3075,d0222r2p240200f0027,0,12,26,0.0
3244,d0222r2p240200f0028,1,14,26,7.5
3424,d0222r2p240200f0029,2,16,26,15.0
3596,d0222r2p240200f0030,3,18,24,22.5
3785,d0222r2p240200f0031,4,25,19,30.0
...,...,...,...,...,...
14258,d0222r2p300200f0065,38,124,1,285.0
14682,d0222r2p300200f0066,39,124,1,292.5
15026,d0222r2p300200f0067,40,125,1,300.0
15397,d0222r2p300200f0068,41,125,1,307.5


In [95]:
lview.dtypes()

TypeError: 'Series' object is not callable

In [96]:
lview.dtypes

ImageID                   object
Frames_post_treatment      int64
Yes_yet                    int32
No_yet                     int32
Time_post                float64
dtype: object

In [97]:
s = lview.set_index('Time_post')

In [98]:
s

,ImageID,Frames_post_treatment,Yes_yet,No_yet
Time_post,,,,
0.0,d0222r2p240200f0027,0,12,26
7.5,d0222r2p240200f0028,1,14,26
15.0,d0222r2p240200f0029,2,16,26
22.5,d0222r2p240200f0030,3,18,24
30.0,d0222r2p240200f0031,4,25,19
...,...,...,...,...
285.0,d0222r2p300200f0065,38,124,1
292.5,d0222r2p300200f0066,39,124,1
300.0,d0222r2p300200f0067,40,125,1


In [99]:
s.loc[0]

,ImageID,Frames_post_treatment,Yes_yet,No_yet
Time_post,,,,
0.0,d0222r2p240200f0027,0,12,26
0.0,d0222r2p270300f0027,0,22,51
0.0,d0222r2p300200f0027,0,20,40


In [100]:
s.loc[0].sum()

ImageID                  d0222r2p240200f0027d0222r2p270300f0027d0222r2p...
Frames_post_treatment                                                    0
Yes_yet                                                                 54
No_yet                                                                 117
dtype: object

In [101]:
s.loc[60].sum()

ImageID                  d0222r2p240200f0035d0222r2p270300f0035d0222r2p...
Frames_post_treatment                                                   24
Yes_yet                                                                178
No_yet                                                                  43
dtype: object

In [102]:
s.loc[120].sum()

ImageID                  d0222r2p240200f0043d0222r2p270300f0043d0222r2p...
Frames_post_treatment                                                   48
Yes_yet                                                                268
No_yet                                                                  14
dtype: object

In [103]:
s.loc[180].sum()

ImageID                  d0222r2p240200f0051d0222r2p270300f0051d0222r2p...
Frames_post_treatment                                                   72
Yes_yet                                                                324
No_yet                                                                   5
dtype: object

In [104]:
s.loc[240].sum()

ImageID                  d0222r2p240200f0059d0222r2p270300f0059d0222r2p...
Frames_post_treatment                                                   96
Yes_yet                                                                361
No_yet                                                                   4
dtype: object

In [105]:
s.loc[300].sum()

ImageID                  d0222r2p240200f0067d0222r2p270300f0067d0222r2p...
Frames_post_treatment                                                  120
Yes_yet                                                                378
No_yet                                                                   1
dtype: object

In [1]:
from datetime import date
import os
import pandas as pd
import math
import os
import statistics
from joblib import Parallel, delayed
from scipy import stats
import plotly.express as px
import single_cell_reloc_parquet.global_functions.global_variables as gv
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotnine
plotly.io.kaleido.scope.mathjax = None #* This is an important line for timely svg output. This is to stop a call to the internet for prettier looking math
import kaleido
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.set_context('paper')

print(plotly.__version__, kaleido.__version__)

# sns.set(rc = {'figure.figsize':(15,8)}) #* Set the figure size to be larger
# sns.set(rc = {'figure.figsize':(40,30)}) #* Set the figure size to be the shape of a powerpoint slide

5.13.0 0.2.1
